In [ ]:
import sys
import pathlib
module_path = str(pathlib.Path().absolute()).replace('/notebooks', '')
sys.path.append(module_path)

In [ ]:
import pickle
import numpy as np
from scripts.jobs.dataset_creation import ReadRasterFile, CreateStridedArray, PreProcessPopulationRaster, StoreDataHDF5

raster_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_rst.tif'

a = ReadRasterFile(raster_fn)
a = PreProcessPopulationRaster(a)
train, test = CreateStridedArray(a)
StoreDataHDF5(train, 'pop_train')
StoreDataHDF5(test, 'pop_test')


In [ ]:

from scripts.jobs.dataset_creation import CreateTFDatasetFromGenerator
from scripts.jobs.model_training import TrainModel

train_dataset = CreateTFDatasetFromGenerator('pop_train')
test_dataset = CreateTFDatasetFromGenerator('pop_test')
model = TrainModel(train_dataset, test_dataset)